In [4]:
import requests
import time
import json
import csv
from bs4 import BeautifulSoup

In [14]:
import mysql.connector
import random
from faker import Faker
from snowflake import SnowflakeGenerator, Snowflake
from datetime import datetime

db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

In [69]:
connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

user_id = 1450492617670998016

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
}

In [9]:
fake = Faker('vi_VN')
sf = Snowflake.parse(856165981072306191, 1288834974657)
gen = SnowflakeGenerator.from_snowflake(sf)

Tạo cửa hàng:

In [10]:
city = ["HNI", "HCM", "DNG", "AGG", "BGG", "BKN", "BLU", "BNH", "VTU",
"BTE", "BDH", "BDG", "BTN", "CMU", "DLK", "DNI", "DTP", "GLI",
"HDG", "HNM", "HTH", "HUG", "HBH", "HYN", "KHA", "KGG", "KTM",
"LCU", "LDG", "LSN", "LAN", "NDH", "NAN", "NBH", "NTN", "CTO",
"HPG", "PTO", "PYN", "QBH", "QNM", "QNI", "QNH", "QTI", "STG",
"SLA", "TNH", "TBH", "TNN", "THA", "TTH", "TGG", "TVH", "TQG",
"VLG", "VPC", "YBI" ]

url = "https://api-crownx.winmart.vn/mt/api/web/v1/store-by-province"


In [12]:
# tao bang category
storeAll = []

for i in city:
    page = 1
    while True:
        params = {
            "PageNumber": page,
            "PageSize": 30,
            "ProvinceCode": i
        }
        page += 1
        
        res = requests.get(url, params)
        res.raise_for_status()
        data = res.json()
        
        districts = data.get("data", [])
        for district in districts:
            wardStores = district.get("wardStores", [])
            for ward in wardStores:
                stores = ward.get("stores", [])
                for store in stores:
                    store_info = {
                        "id": next(gen),
                        "name": store.get("storeName"),
                        "address": store.get("officeAddress"),
                        "provinceName": store.get("provinceName"),
                        "provinceCode": store.get("provinceCode"),
                        "status": store.get("isOpen"),
                    }
                    
                    # check trùng name
                    if not any(c["name"] == store_info["name"] for c in storeAll):
                        storeAll.append(store_info)
                        
        paging = data.get("paging", {})

        if page >= paging.get("totalPages", 0):
                break
        page += 1
        
        time.sleep(0.2)  # tránh spam server quá nhanh
        print(f"Số cuawr hang lấy được: {len(storeAll)}")
        for c in storeAll[:10]:
            print(c)


In [19]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

for store in storeAll:
    id = store["id"]
    name = store["name"]
    email = fake.email()
    address = store["address"]
    description = fake.sentence(nb_words=20)
    create_by = user_id
    create_at = fake.date_time_between(start_date="-2y", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    update_by = user_id
    update_at = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    status = store["status"]
    province_code = store["provinceCode"]
    province_name = store["provinceName"]
    
    sql = """ INSERT INTO store (id, name, email, address, description, create_by, create_at, update_by, update_at, status, province_code, province_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.execute(sql, (id, name, email, address, description, create_by, create_at, update_by, update_at, status, province_code, province_name))

connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Da insert data vao table store")

Da insert data vao table store


Chỗ này là bắt đầu tạo warehouse

In [20]:
import requests
from bs4 import BeautifulSoup

In [31]:
supplierAll = []

def parse_page(page):
    url = f"https://timnhaphanphoi.vn/doanh-nghiep?page={page}"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    items = soup.find_all("div", class_="item")
    for item in items:
        # tên + link
        h3_tag = item.find("h3")
        name = h3_tag.get_text(strip=True) if h3_tag else ""
        web = h3_tag.find("a")["href"] if h3_tag and h3_tag.find("a") else ""
        
        # mô tả
        desc = item.find("div", class_="desc")
        description = desc.get_text(strip=True) if desc else ""
        
        # thông tin chi tiết
        info = item.find("ul", class_="info")
        if info:
            location = info.find("i", class_="fa-map-marker")
            area = ""
            if location:
                raw_location = location.find_parent().get_text(strip=True)
                area = raw_location.split(":")[-1].strip()

            # Loại hình
            type_ = info.find("i", class_="fa-tag")
            type = ""
            if type_:
                raw_type = type_.find_parent().get_text(strip=True)
                type = raw_type.split(":")[-1].strip()

            # Địa chỉ
            address_ = info.find("i", class_="fa-map-o")
            address = ""
            if address_:
                raw_address = address_.find_parent().get_text(strip=True)
                address = raw_address.split(":")[-1].strip()
        else:
            area = type = address = ""
        
        supplier_info = {
            "area": area,
            "name": name,
            "type": type,
            "description": description,
            "address": address,
            "area": area,
            "web": web
        }
        
        if not any(c["name"] == supplier_info["name"] for c in supplierAll):
            supplierAll.append(supplier_info)
        
    print(len(supplierAll))

for i in range(1, 14):
    parse_page(i)

20
40
60
79
99
119
138
157
177
196
215
235
237


In [33]:
fake = Faker("vi_VN")

db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

for sup in supplierAll:
    id = next(gen)
    name = sup["name"]
    email = fake.company_email()
    address = sup["address"]
    description = sup["description"]
    create_by = user_id
    create_at = fake.date_time_between(start_date="-2y", end_date='now').strftime('%Y-%m-%d %H:%M:%S')
    update_by = user_id
    update_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    status = 1
    web = sup["web"]
    type = sup["type"]
    area = sup["area"]
    phone = fake.phone_number()

    sql = """
    INSERT INTO supplier
    (id, name, email, address, description, create_by, create_at, update_by, update_at, status, web, type, area, phone)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    cursor.execute(sql, (id, name, email, address, description, create_by, create_at, update_by, update_at, status, web, type, area, phone))

connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Da insert data vào table supplier")


Da insert data vào table supplier


Tạo bảng Warehouse

In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def crawl_page(page):
    url = f"https://www.yellowpages.vn/cls/486112/kho-bai-dich-vu-kho-bai.html?page={page}"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    boxes = soup.find_all("div", class_="rounded-4 border bg-white shadow-sm mb-3 pb-4")
    print(f"Page {page}: tìm được {len(boxes)} item(s)")

    data = []
    for box in boxes:
        # tên công ty
        name = box.find("h2").get_text(strip=True) if box.find("h2") else ""

        # address
        addr_tag = box.find("i", class_="fa-location-arrow")
        address = addr_tag.find_parent().get_text(strip=True) if addr_tag else ""

        # phone
        phone_tag = box.find("i", class_="fa-phone")
        phone = phone_tag.find_parent().get_text(strip=True) if phone_tag else ""

        # email
        email_tag = box.find("i", class_="fa-envelope")
        email = email_tag.find_parent().get_text(strip=True) if email_tag else ""

        # website
        web_tag = box.find("i", class_="fa-globe")
        website = web_tag.find_parent().get_text(strip=True) if web_tag else ""

        # description (nếu có)
        desc_block = box.find("div", class_="text_quangcao")
        description = desc_block.get_text(" ", strip=True) if desc_block else ""

        # image (lấy ảnh đầu tiên)
        img_tag = box.find("img")
        image = img_tag["src"] if img_tag else ""

        data.append({
            "name": name,
            "address": address,
            "phone": phone,
            "email": email,
            "website": website,
            "description": description,
            "image": image
        })
    return data


# Khởi tạo driver 1 lần
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

all_data = []
for i in range(1, 10):  # test crawl 2 trang
    all_data.extend(crawl_page(i))

driver.quit()

print("Tổng records:", len(all_data))
print(all_data[0])  # in thử 1 record


Page 1: tìm được 45 item(s)
Page 2: tìm được 45 item(s)
Page 3: tìm được 45 item(s)
Page 4: tìm được 45 item(s)
Page 5: tìm được 45 item(s)
Page 6: tìm được 45 item(s)
Page 7: tìm được 45 item(s)
Page 8: tìm được 45 item(s)
Page 9: tìm được 28 item(s)
Tổng records: 388
{'name': 'PCL - Công Ty Cổ Phần Vận Tải Biển Container Thái Bình Dương', 'address': 'Số 99 Võ Chí Công, Phường Xuân Đỉnh,Hà Nội', 'phone': '(024) 32005711', 'email': 'pcl@pacificcontainer.com.vn', 'website': 'www.pacificcontainer.com.vn', 'description': 'Vận Tải Biển Container Thái Bình Dương (Pacific Container Lines - PCL) chuyên\n cung cấp Dịch vụ kho bãi gồm: Quản lý xuất nhập, kiểm kê, phân phối, xếp dỡ, đóng gói bao bì, quản lý hàng hóa, báo cáo tồn kho, kiểm đếm,.. >>> Hệ thống kho hơn 230.000m2 >>> Hàng hóa được bảo hiểm an toàn, phòng cháy chữa cháy >>> Đặc biệt, chúng tôi là thành viên của các hiệp hội gồm: Amcham, Eurocham, Korcham, VLA, WCA. >>> Thương hiệu khách hàng phục vụ: Unilever, Coca-Cola, LG Electroni

In [67]:
import random

services = [
    "Cung cấp dịch vụ cho thuê kho bãi",
    "Chuyên giải pháp lưu trữ và quản lý hàng hóa",
    "Hệ thống kho vận logistics trọn gói",
    "Dịch vụ cho thuê kho tự quản",
    "Cung cấp kho hàng thương mại điện tử"
]

locations = [
    "tại vị trí chiến lược gần các tuyến quốc lộ chính",
    "tọa lạc tại trung tâm KCN lớn",
    "với vị trí đắc địa gần cảng biển",
    "có mạng lưới kho bãi khắp các tỉnh thành",
    "nằm ở khu vực ngoại thành với sân bãi rộng"
]

features = [
    "đảm bảo an ninh với camera giám sát và bảo vệ 24/7",
    "với hệ thống PCCC tự động đạt chuẩn",
    "có nền bê tông chịu lực và trần cao thoáng mát",
    "trang bị phần mềm quản lý kho WMS hiện đại",
    "cho phép xe container ra vào dễ dàng"
]

commitments = [
    "Cam kết mang đến giải pháp tối ưu với chi phí hợp lý.",
    "Chúng tôi đảm bảo sự an toàn tuyệt đối cho hàng hóa của bạn.",
    "Luôn sẵn sàng phục vụ với đội ngũ chuyên nghiệp và tận tâm.",
    "Uy tín - Chất lượng - Hiệu quả là phương châm của chúng tôi."
]

def generate_fake_description():
    """Tạo ra một câu mô tả ngẫu nhiên cho kho."""
    service = random.choice(services)
    location = random.choice(locations)
    feature = random.choice(features)
    commitment = random.choice(commitments)
    
    description = f"{service} {location}, {feature}. {commitment}"
    return description


In [70]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

for ware in all_data:
    id = next(gen)
    name = ware["name"]
    email = fake.email()
    address = ware["address"]
    description = generate_fake_description()
    type = None
    status = 1
    create_by = user_id
    create_at = fake.date_time_between(start_date="-2y", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    update_by = user_id
    update_at = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    image = ware["image"]
    phone = fake.phone_number()
    
    sql = """ INSERT INTO  warehouse (id, name, email, address, description, type, status, create_by,
        create_at, update_by, update_at, image, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    cursor.execute(sql, (id, name, email, address, description, type, status, create_by,
        create_at, update_by, update_at, image, phone))
    
connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Da insert data vao table warehouse")

Da insert data vao table warehouse
